In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
plt.plot?
import requests as rq
import numpy as np
from watson_developer_cloud import PersonalityInsightsV3
from __future__ import print_function
import json
from os.path import join, dirname


def cultOfPersonality(text):
    personality_insights = PersonalityInsightsV3(version='...',
                                             username='...',
                                             password='...', 
                                             url='https://gateway-fra.watsonplatform.net/personality-insights/api')
    profile = personality_insights.profile(
        text,
        content_type='text/plain',
        consumption_preferences=False)
    
    return profile

def cultOfWikipedia(storeOrInput):
    request = {'titles':storeOrInput}
    r = rq.get('https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro&explaintext&redirects=2', params=request).json()
    c_page = list(r['query']['pages'].keys())[0]
    intro = r['query']['pages'][c_page]['extract']
    
    return intro

def produceVector(profile):
    
    vector = []
    for trait in profile['personality']:
        vector.append(trait['percentile'])
    for trait in profile['needs']:
        vector.append(trait['percentile'])  
    for trait in profile['values']:
        vector.append(trait['percentile'])
    
    return vector
    

    
stores = ['Malmo', 'Dubai', 'Shanghai', 'Rio de Janeiro', 'Disneyland Paris', 'Palermo', 'Los Angeles']

user_input = input('Please type in a ToyCo keyword : ')
userIntro = cultOfWikipedia(user_input)
userProfile = cultOfPersonality(userIntro)
userVector = produceVector(userProfile)

our_dict = {}

for store in stores:
    with open(store + '.txt', 'r') as myfile:
        intro=myfile.read().replace('\n', '')
    #intro = cultOfWikipedia(store)
    profile = cultOfPersonality(intro)
    vector = produceVector(profile)
        
    our_dict[store] = np.inner(np.asarray(vector), np.asarray(userVector))
        
x = list(our_dict.values())
y = np.exp(x - np.max(x)) / np.sum(np.exp(x - np.max(x)))

for i, store in enumerate(stores):
    our_dict[store] = y[i]


labels = list(our_dict.keys())
scores = list(our_dict.values())
explode = (0.01,)*7

fig1, ax1 = plt.subplots()
ax1.pie(scores, explode=explode, labels=labels, autopct='%1.1f%%',
        startangle=90)
ax1.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.

plt.show()